In [33]:
from gpxplotter import read_gpx_file, create_folium_map, add_segment_to_map
import gpxplotter
import numpy as np

def extract_from_gpx(gpx_file:str):

    for track in read_gpx_file(gpx_file):
        # extract all gpx data for plotting
        segment = track['segments'][0]
        print((segment.keys()))
        print(np.mean(segment['Velocity / km/h']))

        # extract metrics from the gpx file
        metrics = {
            'distance':segment['distance'][-1],

            'elevation_gain':segment['elevation-up']
        }


    return segment, metrics



the_map = create_folium_map()
segment1, metrics1 = extract_from_gpx('test_route_1.gpx')
add_segment_to_map(the_map, segment1)
#the_map


# for track in read_gpx_file(gpx_file):
#     for i, segment in enumerate(track['segments']):
#         print(segment.keys())
#         add_segment_to_map(the_map, segment)
#         print((segment['distance']))
#         print((segment['elapsed-time']))
#         print((segment['pace']))
#         print(segment['elevation-up'])
#         l = gpxplotter.gpxread.get_distances(segment['lat'], segment['lon'])
# print((l[-1]))
# the_map
# for track in read_gpx_file('test_route_2.gpx'):
#     for i, segment in enumerate(track['segments']):
#         add_segment_to_map(the_map, segment)
#         l = gpxplotter.gpxread.get_distances(segment['lat'], segment['lon'])
# print((l[-1]))
# # To display the map in a Jupyter notebook:
# the_map

dict_keys(['lat', 'lon', 'elevation', 'time', 'latlon', 'elapsed-time', 'distance', 'Distance / km', 'velocity', 'Velocity / km/h', 'pace', 'velocity-level', 'elevation-up', 'elevation-down'])
12.573032890272232


In [41]:
# trying to make a function out of it

import altair as alt
import json
import folium
from gpxplotter import create_folium_map, read_gpx_file, add_segment_to_map
import numpy as np
import base64
from altair_saver import save
import pandas as pd

line_options = {'weight': 8}


the_map = create_folium_map(tooltip = 'This tooltip will appear on hover')

def visualise_gpx(the_map, filename, segment_name = 'Bike Ride'):

    for track in read_gpx_file(filename):
        for i, segment in enumerate(track['segments']):
            add_segment_to_map(the_map, segment,
                            cmap='viridis', line_options=line_options)

    # Create a chart using Altair
    idx = np.argmax(segment['elevation'])

    data = {
        'x': segment['Distance / km'],
        'y': segment['elevation'],
    }
    # Convert the data to a Pandas DataFrame
    df = pd.DataFrame(data)

    # Specify the data type for the x encoding field
    line = alt.Chart(df).mark_line().encode(
        x=alt.X('x', title='Distance / km'),
        y=alt.Y('y', title='Elevation / m')
    )

    WIDTH = 400
    HEIGHT = 200

    line = line.properties(
        width=WIDTH,
        height=HEIGHT
    )

    # Save the chart as a PNG image
    #png_bytes = save(line, format='png')
    line.save('test.html')
    # Encode the PNG image as base64 string
    chart_html = open("test.html", "r").read()
    # Create the HTML content for the popup
    html = ''' <h1 style="font-family: Verdana"> {0}</h1><br>
            <p style="font-family: Verdana"> Distance: {1} </p>
            <p style="font-family: Verdana"> Total elevation Gain: {2} </p>
            <p style="font-family: Verdana"> Average Speed: {3} </p>
            <img src="data:image/png;base64,{4}" />

            <br>
            {4}
            '''.format(segment_name,str(segment['distance'][-1]/1000)[:5]+' km', str(segment['elevation-up'])[:4] +' m', str(np.mean(segment['Velocity / km/h']))[:4] + ' km/h', chart_html)

    iframe = folium.IFrame(html=html, width=500, height=500)
    popup = folium.Popup(iframe, width=500, height=500)

    folium.TileLayer('stamenterrain').add_to(the_map)

    marker = folium.Marker(
        location=segment['latlon'][idx],
        popup=popup,
        icon=folium.Icon(icon='star'),
    )
    marker.add_to(the_map)

    # To store the map as a HTML page:
visualise_gpx(the_map, 'day_1.gpx')
visualise_gpx(the_map, 'day_2.gpx')
visualise_gpx(the_map, 'day_3.gpx')
visualise_gpx(the_map, 'day_4.gpx')
# To display the map in a Jupyter notebook:
the_map

